# Tutorial 6: Hyperparameter Optimization with ToolBrain

This tutorial demonstrates a creative and powerful application of ToolBrain: using reinforcement learning to perform Hyperparameter Optimization (HPO). We will train an agent to find a good hyperparameter for a LightGBM model, showcasing how ToolBrain can be used for optimization tasks.

This tutorial is based on the examples in `examples/02_lightgbm_hpo_training_with_grpo/` and `examples/04_lightgbm_hpo_training_with_dpo/`.

## The Concept: HPO as an RL Problem

We can frame HPO as a reinforcement learning problem:

-   **Agent**: An LLM-based agent.
-   **Action**: The agent's action is to choose a value for a hyperparameter and call a function with it.
-   **Environment**: The environment is a function that takes the hyperparameter, trains a model, and returns a performance score (e.g., accuracy).
-   **Reward**: The performance score *is* the reward.

By training the agent to maximize its reward, we are effectively teaching it to discover hyperparameters that lead to better model performance.

## Step 1: The Environment as a Tool

Our environment is encapsulated in a single tool defined in `lightgbm_model.py`:

In [9]:
# pip install lightgbm scikit-learn

In [1]:
import lightgbm as lgb
import sklearn.datasets
import sklearn.model_selection
import sklearn.metrics
from smolagents import tool

@tool
def run_lightgbm(feature_fraction: float) -> float:
    """
    Train and evaluate a LightGBM binary classifier on the Breast Cancer dataset.

    This function loads the Breast Cancer dataset from scikit-learn,
    splits it into training and validation sets, trains a LightGBM model
    with the specified feature_fraction parameter, applies early stopping,
    and evaluates prediction accuracy on the validation set.
    Example:
        >>> run_lightgbm(0.8)
        Prediction accuracy 0.96

    Args:
        feature_fraction (float): Fraction of features to use when training
            each boosting iteration. Must be between 0.0 and 1.0.
    """
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)

    # Train/validation split
    train_x, valid_x, train_y, valid_y = sklearn.model_selection.train_test_split(
        data, target, test_size=0.25, random_state=42
    )

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "feature_pre_filter": False,
        "feature_fraction": feature_fraction,
    }

    # Prepare LightGBM datasets
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)

    # Train with early stopping
    gbm = lgb.train(
        param,
        dtrain,
        valid_sets=[dvalid],
        num_boost_round=1000,
        callbacks=[lgb.early_stopping(50, verbose=False)]
    )

    # Predict and evaluate accuracy
    preds = gbm.predict(valid_x, num_iteration=gbm.best_iteration)
    pred_labels = (preds >= 0.5).astype(int)
    accuracy = sklearn.metrics.accuracy_score(valid_y, pred_labels)
    print("Prediction accuracy", accuracy)
    return  accuracy

/home/quan/MyProjects/ToolBrain/ToolBrain/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The agent's task is to call this function. The only choice it has to make is what value to use for the `feature_fraction` argument. The function's return value—the accuracy of the trained LightGBM model—will serve as our reward signal.

## Step 2: A Custom Reward Function

Since the tool's output is our desired reward, we need a custom reward function that captures this value. This function is defined in `run_hpo_training.py` :

In [8]:
from smolagents import CodeAgent, TransformersModel 
from toolbrain import Brain, Trace
from typing import Any


# customised reward function, just return the lightgbm accuracy output, 0 otherwise
def reward_accuracy(trace: Trace, **kwargs: Any) -> float:
    for turn in trace:
        try:
            reward = float(turn["action_output"])  # output of the function run_lightgbm
            return reward
        except:
            # failed to call the function set reward to 0
            reward = 0.0
        return reward

This function directly translates the `action_output` (the return value of `run_lightgbm` -  the accuracy when training the lightgbm model on the dataset) into the reward for the RL algorithm.

## Step 3: The Training Query

The query we give the agent is open-ended. We are not asking for a specific answer, but rather encouraging exploration.

In [10]:
# --- 2. Prepare Training Data ---
training_dataset = [
    {
        "query": "Use the run_lightgbm tool with a suggested value of feature_fraction (must be between 0.0 and 1.0)."
    }
]

## Step 4:  The LLM model to train

We first initiate the LLM model for the code agent

In [11]:
print("📥 Initializing TransformersModel...")
trainable_model = TransformersModel(model_id="Qwen/Qwen2.5-0.5B-Instruct",
                                    max_new_tokens=128)

print("✅ TransformersModel initialized.")

📥 Initializing TransformersModel...
✅ TransformersModel initialized.


This prompt guides the agent to use the correct tool while leaving the crucial choice of the `feature_fraction` value up to it.

## Step 5: Configuring the Brain

We assemble all the pieces in the `Brain` configuration. We provide the agent, the custom reward function, and the learning algorithm.

### 5.1 Training the code agent using GRPO

In the GRPO example, the agent will try different values for `feature_fraction` and be rewarded based on the resulting accuracy. The algorithm will learn to propose values that lead to higher scores.

In [13]:
# From examples/02_lightgbm_hpo_training_with_grpo/run_hpo_training.py

# Create the code agent
print("🔧 Creating CodeAgent...")
my_agent_grpo = CodeAgent(
    tools=[run_lightgbm],
    model=trainable_model,
    max_steps=1
)
print("✅ Agent created.")

brain_grpo = Brain(
    agent=my_agent_grpo,
    reward_func=reward_accuracy,
    algorithm="GRPO"
)

🔧 Creating CodeAgent...
✅ Agent created.


Train the agent using the GRPO algorithm

In [14]:
brain_grpo.train(training_dataset, num_iterations=10)


🚀 Starting training...

--- Iteration 1/10 ---

🔄 Training step for query: 'Use the run_lightgbm tool with a suggested value o...'
Calling get_trace with query: 'Use the run_lightgbm tool with a suggested value o...'
Enable tool retrieval: False
    📝 Trace 1/2


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Use the run_lightgbm tool with a suggested value of feature_fraction (must be between 0.0 and 1.0).             │
│                                                                                                                 │
╰─ TransformersModel - Qwen/Qwen2.5-0.5B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import numpy as np                                                                                               
  from sklearn.datasets import make_classification                                                                 
  from sklearn.model_selection import train_test_split                                                             
  from lightgbm import LGBMClassifier                                                                              
                                                                                                                   
  # Generate synthetic classification dataset                                                                      
  X, y = make_classification(n_samples=1000, n_features=20, n_informative=2, n_redundant=10, random_state=42)      
                                                                                                                   
  # Split dataset into training and test sets                                                                      
  X_train, X                                                                                                       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            To solve this task, we need to use the `run_lightgbm` function with a specific value of 
`feature_fraction`, which is recommended to improve the performance of the model. Here's how I would approach this:

1. Define the value of `feature_fraction`: We'll use the suggestion of 0.36 to increase the model's accuracy.
2. Call the `run_lightgbm` function with the suggested value of `feature_fraction`.
3. Extract and print the returned result to determine the final age of the Pope based on the updated feature 
fraction.

Let's start by defining the value of `feature_fraction`</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 1: Duration 1.99 seconds| Input tokens: 2,102 | Output tokens: 128]

Reached max steps.

[Step 2: Duration 1.85 seconds| Input tokens: 2,603 | Output tokens: 256]

  🎯 Computing rewards for 2 traces...
      Using single-trace reward function
      🎯 Trace 1 Reward: 0.000
      🎯 Trace 2 Reward: 0.000
📈 Sliding window avg reward (last 10): 0.0972
⚠️ All rewards are zero in this batch. Skipping training step to save compute.

🎉 Training finished!


In [21]:
# delete brain_grpo and my_agent_grpo to save memory
del brain_grpo
del my_agent_grpo

### 5.2 Training the code agent using DPO

The setup for DPO is nearly identical, just changing the algorithm name. DPO will learn by comparing pairs of attempts. For instance, if calling `run_lightgbm(0.7)` yields a higher accuracy than `run_lightgbm(0.5)`, the first trace will be "chosen" and the second will be "rejected."

In [16]:
# Create the code agent
print("🔧 Creating CodeAgent...")
my_agent_dpo= CodeAgent(
    tools=[run_lightgbm],
    model=trainable_model,
    max_steps=1
)
print("✅ Agent  created.")

brain_dpo= Brain(
    agent=my_agent_dpo,
    reward_func=reward_accuracy,
    algorithm="DPO"
)

🔧 Creating CodeAgent...
✅ Agent created.


Train the agent using the DPO algorithm

In [22]:
# From examples/04_lightgbm_hpo_training_with_dpo/run_hpo_training.py

brain_dpo.train(training_dataset, num_iterations=10)


🚀 Starting training...

--- Iteration 1/10 ---

🔄 Training step for query: 'Use the run_lightgbm tool with a suggested value o...'
Calling get_trace with query: 'Use the run_lightgbm tool with a suggested value o...'
Enable tool retrieval: False
    📝 Trace 1/2


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Use the run_lightgbm tool with a suggested value of feature_fraction (must be between 0.0 and 1.0).             │
│                                                                                                                 │
╰─ TransformersModel - Qwen/Qwen2.5-0.5B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            ```python
# Function to run the LightGBM algorithm
import lightgbm as lgb

def run_lightgbm(feature_fraction):
    # Load the Breast Cancer dataset
    X_train, y_train, X_test, y_test = load_breast_cancer()

    # Split the data into train and test sets
    X_train, X_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Create a LightGBM model with the suggested feature_fraction parameter
    model = lgb.LGBMClassifier(feature_fraction=feature_fraction, n</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 1: Duration 2.00 seconds| Input tokens: 2,102 | Output tokens: 128]

...

Error in code parsing:
Your code snippet is invalid, because the regex pattern <code>(.*?)</code> was not found in it.
            Here is your code snippet:
            Sure, let's break down the task step-by-step:

1. We need to use the `run_lightgbm` function with a suggested value of `feature_fraction`.
2. The function takes two arguments: `feature_fraction`, which is a float representing the fraction of features to 
use when training each boosting iteration.
3. We'll test different values of `feature_fraction` within the range [0.0, 1.0\] to see which one gives the 
highest prediction accuracy.
4. Once we identify the optimal `feature_fraction`, we'll apply it to train a LightGBM model and calculate the 
prediction accuracy.

Let</code>
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            <code>
            # Your python code here
            </code>
Make sure to provide correct code blobs.

[Step 1: Duration 2.02 seconds| Input tokens: 2,102 | Output tokens: 128]

Reached max steps.

[Step 2: Duration 1.34 seconds| Input tokens: 2,603 | Output tokens: 219]

  🎯 Computing rewards for 2 traces...
      Using single-trace reward function
      🎯 Trace 1 Reward: 0.000
      🎯 Trace 2 Reward: 0.000
📈 Sliding window avg reward (last 10): 0.0000
⚠️ All rewards are zero in this batch. Skipping training step to save compute.

🎉 Training finished!


## The Outcome

As you run the training script, you should see the `Sliding window avg reward` gradually increase. This indicates that the agent is getting better at suggesting `feature_fraction` values that result in higher model accuracy. You are watching an LLM learn to perform hyperparameter optimization through reinforcement learning.

---

This example illustrates the flexibility of ToolBrain. By creatively defining the "tools" and "rewards," you can adapt the framework to solve a range of optimization and exploration problems beyond simple question-answering.